In [1]:
import os
import sys
import re
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

import api


/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meta_df = pd.read_csv("../../intermediate_data/cpc/metadata.csv")

In [3]:
meta_df['agenda_pages'] = (meta_df['filename']=='agenda')*(meta_df['pages'])
meta_df['supplemental_pages'] = (meta_df['filename']=='supplemental-docs')*(meta_df['pages'])
meta_df['minutes_pages'] = (meta_df['filename']=='minutes')*(meta_df['pages'])
meta_df2 = meta_df.groupby(['year','date']).agg(
    agenda_pages = ('agenda_pages','sum'),
    supplemental_pages = ('supplemental_pages','sum'),
    minutes_pages = ('minutes_pages','sum')
).reset_index()
meta_df2['has_all'] = (meta_df2['agenda_pages']>0) & (meta_df2['supplemental_pages']>0) & (meta_df2['minutes_pages']>0)
meta_df2['total_pages'] = meta_df2['agenda_pages'] + meta_df2['supplemental_pages'] + meta_df2['minutes_pages']

In [6]:
idx = meta_df2['has_all']
n_meetings = len(meta_df2.loc[idx])
n_years = len(meta_df2.loc[idx,'year'].unique())
min_year = meta_df2.loc[idx,'year'].astype('int').min()
max_year = meta_df2.loc[idx,'year'].astype('int').max()
n_pages = meta_df2.loc[idx, 'total_pages'].sum()

print(f"{n_meetings} meetings")
print(f"across {n_years} years from {min_year} to {max_year}")
print(f"totaling {n_pages:,g} pages of documents")

150 meetings
across 7 years from 2018 to 2024
totaling 23,430 pages of documents
